In [ ]:
import os
import glob
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim

from PIL import Image
from monai.networks.nets import DenseNet121
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import functional as TF
from monai.transforms import (
    Compose, LoadImage, Resize, ScaleIntensity, ToTensor
)

### Setting the device - cuda / cpu

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Dataset base path
base_dir = 'dataset'
train_dir = os.path.join(base_dir, 'training')
test_dir = os.path.join(base_dir, 'testing')

### Defining class names and index

In [4]:
# Define class names and index
class_names = sorted(os.listdir(train_dir))
class_to_idx = {cls_name: i for i, cls_name in enumerate(class_names)}
num_classes = len(class_names)

In [ ]:
# Get image paths and labels
def get_image_paths_and_labels(folder):
    image_paths = []
    labels = []
    for cls_name in class_names:
        cls_folder = os.path.join(folder, cls_name)
        files = glob.glob(os.path.join(cls_folder, '*.jpg'))
        image_paths.extend(files)
        labels.extend([class_to_idx[cls_name]] * len(files))
    return image_paths, labels

train_paths, train_labels = get_image_paths_and_labels(train_dir)
test_paths, test_labels = get_image_paths_and_labels(test_dir)

### MONAI-style transforms

In [6]:
# MONAI-style transforms
transform = Compose([
    Resize((224, 224)),
    ScaleIntensity(),
    ToTensor()
])

### Custom dataset

In [ ]:
# Custom dataset
class BrainTumorDataset(Dataset):
    def __init__(self, image_paths, labels, transform):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load as grayscale
        image = Image.open(self.image_paths[idx]).convert('L')
        image = TF.to_tensor(image)  # shape: [1, H, W]
        image = self.transform(image)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return image, label

### Datasets and loaders

In [8]:
# Datasets and loaders
train_ds = BrainTumorDataset(train_paths, train_labels, transform)
test_ds = BrainTumorDataset(test_paths, test_labels, transform)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=16)

### Model architecture

In [9]:
# Model
model = DenseNet121(spatial_dims=2, in_channels=1, out_channels=num_classes).to(device)

# Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

### Prepare lists to store data

In [10]:
# Declare the lists
epochs_num_list = []
loss_list = []
accuracy_list = []

In [ ]:
# Training loop
n_epochs = 20
for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        print(f'Total: {total} | Corect: {correct}')

    accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%')

    # Store result data
    epochs_num_list.append(epoch + 1)
    loss_list.append(epoch_loss)
    accuracy_list.append(accuracy)

# Save model
torch.save(model.state_dict(), f'monai_tumor_classifier_121_densenet_{n_epochs}_epochs.pth')

### Store results data in a DataFrame

In [ ]:
results_df = pd.DataFrame({
    'Epoch': epochs_num_list,
    'Loss': loss_list,
    'Accuracy': accuracy_list,
})

### Save results in a CVS file

In [ ]:
results_df.to_csv(f'training_{n_epochs}_epochs.csv', index = False)